In [1]:
import ee
import geemap.core as geemap

!pip install zarr
!pip install xarray
!pip install xarray[complete]
!pip install boto3
!pip install minio
!pip install nest-asyncio
import zarr
import xarray as xr
import pandas as pd
import numpy as np
import boto3
import io
import minio
import os
import asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.4/184.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 kB 1.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.8/72.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.4/117.4 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.2 MB/s eta 0

In [2]:
ee.Authenticate()
ee.Initialize(project='planar-osprey-377213')

In [3]:
AWS_S3_ENDPOINT_URL = f'b012-171-243-49-191.ngrok-free.app'

s3 = minio.Minio(
  AWS_S3_ENDPOINT_URL,
  "GK5f748179ea519bae8429ebd1",
  "d981f2ff47b0d14aa842aa061c934dcd052fed4985e791ed0f793131ded7e2ec",
  # Force the region, this is specific to garage
  region="garage",
)

response = s3.list_buckets()
print(response)

[Bucket('era-bucket')]


In [4]:
# Open the Zarr dataset
ds = xr.open_zarr("gs://gcp-public-data-arco-era5/ar/1959-2022-full_37-6h-0p25deg_derived.zarr")

# Define filtering parameters
start_year = "2018-01-01"
end_year = "2018-01-05"
selected_levels = [50, 100, 150, 200, 250, 300, 400, 500, 600, 700, 850, 925, 1000]

# Filter dataset
filtered_ds = ds.sel(
    time=slice(start_year, end_year),  # Filter by time range
    level=selected_levels  # Filter by pressure levels
)

# Resample to 128x256 spatial resolution
resampled_ds = filtered_ds.interp(
    latitude=pd.Series(np.linspace(filtered_ds.latitude.min(), filtered_ds.latitude.max(), 128)),
    longitude=pd.Series(np.linspace(filtered_ds.longitude.min(), filtered_ds.longitude.max(), 256)),
    method='nearest'
)

# Ensure that specific variables are included
resampled_ds = resampled_ds[['10m_u_component_of_wind', '10m_v_component_of_wind',
                            '2m_temperature', 'mean_sea_level_pressure',
                            'geopotential', 'specific_humidity',
                            'u_component_of_wind', 'v_component_of_wind',
                            'temperature']]
resampled_ds

<xarray.Dataset> Size: 181MB
Dimensions:                  (time: 20, latitude: 128, longitude: 256, level: 13)
Coordinates:
  * level                    (level) int64 104B 50 100 150 200 ... 850 925 1000
  * time                     (time) datetime64[ns] 160B 2018-01-01 ... 2018-0...
  * latitude                 (latitude) float64 1kB -90.0 -88.58 ... 88.58 90.0
  * longitude                (longitude) float64 2kB 0.0 1.411 ... 358.3 359.8
Data variables:
    10m_u_component_of_wind  (time, latitude, longitude) float32 3MB dask.array<chunksize=(1, 128, 256), meta=np.ndarray>
    10m_v_component_of_wind  (time, latitude, longitude) float32 3MB dask.array<chunksize=(1, 128, 256), meta=np.ndarray>
    2m_temperature           (time, latitude, longitude) float32 3MB dask.array<chunksize=(1, 128, 256), meta=np.ndarray>
    mean_sea_level_pressure  (time, latitude, longitude) float32 3MB dask.array<chunksize=(1, 128, 256), meta=np.ndarray>
    geopotential             (time, level, latitude, longitude) float32 34MB dask.array<chunksize=(1, 13, 128, 256), meta=np.ndarray>
    specific_humidity        (time, level, latitude, longitude) float32 34MB dask.array<chunksize=(1, 13, 128, 256), meta=np.ndarray>
    u_component_of_wind      (time, level, latitude, longitude) float32 34MB dask.array<chunksize=(1, 13, 128, 256), meta=np.ndarray>
    v_component_of_wind      (time, level, latitude, longitude) float32 34MB dask.array<chunksize=(1, 13, 128, 256), meta=np.ndarray>
    temperature              (time, level, latitude, longitude) float32 34MB dask.array<chunksize=(1, 13, 128, 256), meta=np.ndarray>

In [5]:
def uploadFile(bucket_name, object_name, data):
    npy_buffer = io.BytesIO()
    np.save(npy_buffer, data)
    npy_buffer.seek(0)  # Move to the start of the buffer

    s3.put_object(
        bucket_name=bucket_name,
        object_name=object_name,
        data=npy_buffer,
        length=npy_buffer.getbuffer().nbytes,
        content_type="application/octet-stream"
    )

# Test upload
# uploadFile("era-bucket", "tmp.npy", np.array([1, 2, 3]))

In [34]:
from functools import partial


single_level_vnames = ["u10", "v10", "t2m", "msl"]
multi_level_vnames = ["z", "q", "u", "v", "t"]
height_level = [50, 100, 150, 200, 250, 300, 400, 500, 600, 700, 850, 925, 1000]

single_level_mapping = {
    "10m_u_component_of_wind": "u10",
    "10m_v_component_of_wind": "v10",
    "2m_temperature": "t2m",
    "mean_sea_level_pressure": "msl",
}

multi_level_mapping = {
    "geopotential": "z",
    "specific_humidity": "q",
    "u_component_of_wind": "u",
    "v_component_of_wind": "v",
    "temperature": "t"
}

async def uploadMultiLevel(time):
    time_value = str(resampled_ds.isel(time=time).time.values)
    timestamp = str(time_value).split(".")[0]
    year = pd.Timestamp(time_value).year

    async def uploadKey(vname):
        file = os.path.join(str(year), timestamp).replace("T", "/")

        async def uploadLevel(levelIdx):
            height = height_level[levelIdx]
            data = resampled_ds.isel(time=time)[vname][levelIdx]

            mapped_vname = multi_level_mapping[vname]

            url = f"{file}-{mapped_vname}-{height}.0.npy"

            uploadFile("era-bucket", url, data.values)
            print(f"Upload file {url}")


        await asyncio.gather(*[uploadLevel(i) for i in range(13)])

    await asyncio.gather(*[uploadKey(vname) for vname in multi_level_mapping.keys()])


async def uploadSingleLevel(time):
    time_value = str(resampled_ds.isel(time=time).time.values)
    timestamp = str(time_value).split(".")[0]
    year = pd.Timestamp(time_value).year

    async def uploadKey(vname):
        data = resampled_ds.isel(time=time)[vname]

        mapped_vname = single_level_mapping[vname]

        file = os.path.join("single/", str(year), timestamp).replace("T", "/")
        url = f"{file}-{mapped_vname}.npy"

        uploadFile("era-bucket", url, data.values)
        print(f"Upload file {url}")

    await asyncio.gather(*[uploadKey(vname) for vname in single_level_mapping.keys()])

async def uploadAll(time):
    await asyncio.gather(
        uploadMultiLevel(time),
        uploadSingleLevel(time))

In [35]:
import nest_asyncio
nest_asyncio.apply()


async def main():
    # Run all uploads concurrently
    await asyncio.gather(*[uploadAll(i) for i in range(len(resampled_ds["time"]))])

asyncio.run(main())

Upload file single/2018/2018-01-01/00:00:00-u10.npy
Upload file single/2018/2018-01-01/00:00:00-v10.npy
Upload file single/2018/2018-01-01/00:00:00-t2m.npy
Upload file single/2018/2018-01-01/00:00:00-msl.npy
Upload file single/2018/2018-01-01/06:00:00-u10.npy
Upload file single/2018/2018-01-01/06:00:00-v10.npy
Upload file single/2018/2018-01-01/06:00:00-t2m.npy
Upload file single/2018/2018-01-01/06:00:00-msl.npy
Upload file single/2018/2018-01-01/12:00:00-u10.npy
Upload file single/2018/2018-01-01/12:00:00-v10.npy
Upload file single/2018/2018-01-01/12:00:00-t2m.npy
Upload file single/2018/2018-01-01/12:00:00-msl.npy
Upload file single/2018/2018-01-01/18:00:00-u10.npy
Upload file single/2018/2018-01-01/18:00:00-v10.npy
Upload file single/2018/2018-01-01/18:00:00-t2m.npy
Upload file single/2018/2018-01-01/18:00:00-msl.npy
Upload file single/2018/2018-01-02/00:00:00-u10.npy
Upload file single/2018/2018-01-02/00:00:00-v10.npy
Upload file single/2018/2018-01-02/00:00:00-t2m.npy
Upload file 